In [ ]:
from pathlib import Path
from model_settings import ms
import pandas as pd
import sys
import os
import numpy as np
from model_settings import ms
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.dates as mdates
from datetime import datetime
from datetime import timedelta
import QuantLib as ql
from itertools import product

In [ ]:
os.chdir(str(Path().resolve()))

In [ ]:
store = pd.HDFStore(r'alphaVantage Vanillas.h5')
keys = store.keys()
store.close()
len(keys)

In [ ]:
contract_keys = [key for key in keys if key.find('hottest_contracts')!= -1]
len(contract_keys)

In [ ]:
surface_keys = [key for key in keys if key.find('surface')!= -1]
len(surface_keys)

In [ ]:
raw_data_keys = [key for key in keys if key.find('raw_data')!=-1]
len(raw_data_keys)

# Routine Start

In [ ]:
store.close()

In [ ]:
surface_key = surface_keys[2]
surface_key

## loop start

In [ ]:
second_backslash_pos = surface_key.find('/', 1)
date_key_component = surface_key[:second_backslash_pos+1]
date_key_component

In [ ]:
accomp = [key for key in contract_keys if f'{date_key_component}hottest_contracts' in contract_keys][0]
accomp

In [ ]:
raw_data_key = [key for key in raw_data_keys if f'{date_key_component}raw_data' in raw_data_keys][0]
raw_data_key

In [ ]:
date_string = date_key_component[date_key_component.find('_',0)+1:date_key_component.find('_',0)+11]
date_string

In [ ]:
calculation_datetime = datetime.strptime(date_string,'%Y_%m_%d')
calculation_datetime

In [ ]:
"""
HDF5 collection
"""
while True:
    try:
        store.open()
        raw_data = pd.DataFrame(store[raw_data_key])
        contracts = pd.DataFrame(store[accomp])
        vol_matrix = pd.DataFrame(store[surface_key])
        break
    except OSError:
        print(OSError)
        print('retrying in')
        for i in range (0,5):
            print(5-i)
    finally:
        store.close()

In [ ]:
pd.to_numeric(raw_data['last'])
raw_data['date'] = pd.to_datetime(raw_data['date'])
raw_data['expiration'] = pd.to_datetime(raw_data['expiration'])
raw_data['implied_volatility'] = pd.to_numeric(raw_data['implied_volatility'])
raw_data['strike'] = pd.to_numeric(raw_data['strike'])
raw_data['last'] = pd.to_numeric(raw_data['last'])
raw_data.dtypes

In [ ]:
contract_maturities = np.array((raw_data['expiration'] - raw_data['date']) / timedelta(days=1)).astype(int)
raw_data['days_to_maturity'] = contract_maturities

In [ ]:
vol_matrix

In [ ]:
contracts.iloc[:5,:]

In [ ]:
store.close()

In [ ]:
contracts['spot_price'].unique()

In [ ]:
spot = float(contracts['spot_price'].unique()[0])
spot

In [ ]:
T = vol_matrix.columns.tolist()
K = vol_matrix.index.tolist()
print(f"\nmaturitites:\n     {T}\nstrikes:\n     {K}\n")

In [ ]:
calculation_date = ql.Date(
    calculation_datetime.day,
    calculation_datetime.month,
    calculation_datetime.year)
print(calculation_date)

In [ ]:
r = 0.04
g = 0.018

In [ ]:
s = spot
s

In [ ]:
ql.Settings.instance().evaluationDate = calculation_date
flat_ts, dividend_ts = ms.ql_ts_rg(r, g, calculation_date)
S_handle = ql.QuoteHandle(ql.SimpleQuote(s))

heston_helpers = []
v0 = 0.01; kappa = 0.2; theta = 0.02; rho = -0.75; eta = 0.5;
process = ql.HestonProcess(
    flat_ts,
    dividend_ts,
    S_handle,
    v0,                # Initial volatility
    kappa,             # Mean reversion speed
    theta,             # Long-run variance (volatility squared)
    eta,               # Volatility of the volatility
    rho                # Correlation between asset and volatility
)
model = ql.HestonModel(process)
engine = ql.AnalyticHestonEngine(model)

for t in T:
    for k in K:
        p = ql.Period(int(t),ql.Days)
        volatility = vol_matrix.loc[k,t]
        helper = ql.HestonModelHelper(
            p, ms.calendar, float(s), k, 
            ql.QuoteHandle(ql.SimpleQuote(volatility)), 
            flat_ts, 
            dividend_ts
            )
        helper.setPricingEngine(engine)
        heston_helpers.append(helper)

lm = ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)


model.calibrate(heston_helpers, lm,
                  ql.EndCriteria(1000, 50, 1.0e-8,1.0e-8, 1.0e-8))

theta, kappa, eta, rho, v0 = model.params()
heston_parameters = pd.Series(
    [theta, kappa, eta, rho, v0],
    index = ['theta', 'kappa', 'eta', 'rho', 'v0'],
    dtype = float
)
print(heston_parameters)

# Plotting

In [ ]:
printdate = calculation_datetime.strftime('%A, %Y-%m-%d')
KK,TT = np.meshgrid(K,T)
VV = np.array(
        [[vol_matrix.loc[k,t] for k in K] for t in T]
        )
fig = plt.figure(figsize=plt.figaspect(0.5))

ax1 = fig.add_subplot(1, 2, 1, projection='3d')
surf = ax1.plot_surface(KK, TT, VV, rstride=1, cstride=1, cmap=cm.magma, linewidth=0.1)


ax1.set_zlim(0, float(max(vol_matrix.values.flatten())))
ax1.set_title(f"volatiltiy surface {printdate}")
ax1.set_xlabel('strike')
ax1.set_ylabel('maturity')

# Second subplot for wireframe plot
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
ax2.plot_wireframe(KK, TT, VV, rstride=1, cstride=1,color='black')
ax2.set_zlim(0, float(max(vol_matrix.values.flatten())))
ax2.set_title('wireframe')
ax2.set_xlabel('strike')
ax2.set_ylabel('maturity')
plt.tight_layout()
plt.show()
plt.clf()

# Testing

In [ ]:
contracts.iloc[:5,:]

In [ ]:
calibration_test_data = raw_data.copy()[['strike','type','last','implied_volatility','days_to_maturity']]
calibration_test_data.columns = ['strike_price','w','market_price','volatility','days_to_maturity']
calibration_test_data['spot_price'] = s
calibration_test_data['risk_free_rate'] = r
calibration_test_data['dividend_rate'] = g
calibration_test_data = calibration_test_data[calibration_test_data['days_to_maturity'].isin(contracts['days_to_maturity'])]
calibration_test_data = calibration_test_data[calibration_test_data['days_to_maturity'].isin(contracts['days_to_maturity'])]
calibration_test_data.dtypes

In [ ]:
calibration_test_data

In [ ]:
calibration_test_data[heston_parameters.index.tolist()] = np.tile(heston_parameters,(calibration_test_data.shape[0],1))
calibration_test_data.loc[:,'moneyness'] = ms.vmoneyness(calibration_test_data['spot_price'].values,calibration_test_data['strike_price'].values,calibration_test_data['w'].values)
calibration_test_data['calculation_date'] = calculation_datetime
calibration_test_data['black_scholes'] = ms.vector_black_scholes(calibration_test_data)
calibration_test_data['heston_price'] = ms.vector_heston_price(calibration_test_data)
calibration_test_data.loc[:,'absolute_error'] = calibration_test_data['heston_price'].values - calibration_test_data['black_scholes'].values
plt.figure()
plt.hist(calibration_test_data['absolute_error'],bins=int(calibration_test_data.shape[0]**(1/2)))
plt.xlabel('absolute relative pricing error')
plt.title('distribution of pricing error of Heston against Black Scholes')
plt.show()
plt.clf()

In [ ]:
large_errors = calibration_test_data.copy()[calibration_test_data['absolute_error']>=1]
print(large_errors.describe())

# Bulk Calibration Testing

In [ ]:
from pathlib import Path
import os

In [ ]:
historical_calibration = pd.read_csv(
    os.path.join(
        str(Path().resolve()),'AlphaVantageCalibrated.csv'
    )
).iloc[:,1:].copy()
print(historical_calibration.dtypes)

In [ ]:
historical_calibration['calculation_date'] = pd.to_datetime(
    historical_calibration['calculation_date'],format='%Y-%m-%d')

In [ ]:
print(
    historical_calibration.copy().describe()[[
        'strike_price', 'volatility', 'days_to_maturity',
        'spot_price', 'risk_free_rate', 'dividend_rate', 
        'moneyness', 'black_scholes', 'heston_price','market_price'
    ]].iloc[1:,:]
)
print(f"\ncount: {historical_calibration.shape[0]}")

In [ ]:
historical_heston_parameters = historical_calibration[[
    'calculation_date','theta','kappa','rho','eta','v0']].copy().set_index(
    'calculation_date').drop_duplicates().sort_index(ascending=False)
print(historical_heston_parameters.describe().iloc[1:,:])
print(f"\ncount: {historical_heston_parameters.shape[0]}")

In [ ]:
pd.set_option('display.max_rows',None)
pd.set_option('display.float_format','{:.6f}'.format)
print(historical_heston_parameters)
pd.reset_option('display.max_rows')
pd.reset_option('display.float_format')

In [ ]:
from plotnine import ggplot, aes, geom_point, labs, theme
def heat_scatter(
            df, X_name, Y_name, xlabel, ylabel, runtime, title):
        predictive_performance_plot = (
            ggplot(df, 
                   aes(x=X_name, y=Y_name)) + 
            geom_point(alpha=0.1) + 
            labs(x=xlabel, 
                 y=ylabel,
                 title=title) + 
            theme(legend_position="")
            )
        predictive_performance_plot.show()
        plt.clf()
        return predictive_performance_plot   

In [ ]:
heat_scatters = []
df = historical_heston_parameters.copy().reset_index()
for col in historical_heston_parameters:
    heat_scatters.append(heat_scatter(
            df, 'calculation_date', col, '', col, '', col)
    )